In [1]:
import numpy as np
import tifffile as tiff
from skimage.measure import regionprops, label
from scipy.spatial import distance
from tqdm import tqdm
import pandas as pd
import os
import time

from joblib import Parallel, delayed
import multiprocessing

import gc

In [2]:
##functions
def filter_single_z_slice_objects(array):
    # Step 1: Count unique labels in each z-slice
    unique_labels_all_slices = np.array(0)
    for z in range(array.shape[0]):
        unique_labels_all_slices = np.append(unique_labels_all_slices,np.unique(array[z]))
    
    #Step 2: Identify labels that appear only once across all z-slices
    print("finding labels that are only in 1 z slice")
    unique_values, value_counts = np.unique(unique_labels_all_slices, return_counts=True)
    non_duplicate_values = unique_values[value_counts == 1]

    print(f"Found {len(non_duplicate_values)} 1 slice only labels out of {len(unique_values)} total labels")

    # Step 3: Filter out objects with labels present in only one z-slice
    #print("copying original array")
    filtered_array = np.copy(array)
    # Create a mask where True indicates the positions where 'array' matches any value in 'non_duplicate_values'
    mask = np.isin(array, non_duplicate_values)
    
    # Set the elements in `array` where mask is True to 0
    filtered_array[mask] = 0
    return filtered_array

# Function to calculate IoU between two binary masks (same shape)
def calculate_iou(slice1, slice2):
    intersection = np.logical_and(slice1, slice2).sum()
    union = np.logical_or(slice1, slice2).sum()
    if union == 0:
        return 0
    return intersection / union

# Function to expand slices for a single object
def expand_slices(label_id, z_centroid, y_centroid, x_centroid, membrane_filtered, membrane, z_slices, iou_threshold):
    current_object = membrane_filtered[z_centroid] == label_id
    orig_label_id = membrane[z_centroid, y_centroid, x_centroid]
    
    if orig_label_id != 0:
        expanded_slices = [(z_centroid, orig_label_id)]  # Start with the centroid slice
    else:
        expanded_slices = []
    
    # Expand upwards and downwards in Z
    for direction in [-1, 1]:  # -1 for upwards, 1 for downwards
        z = z_centroid + direction
        
        while 0 <= z < z_slices:  # Ensure z is within bounds
            candidate_slice = membrane[z]  # Get the original membrane slice at z
            candidate_object_value = candidate_slice[y_centroid, x_centroid]
            
            if candidate_object_value != 0:  # If object exists in this slice
                candidate_object = candidate_slice == candidate_object_value
                iou_score = calculate_iou(current_object, candidate_object)
                
                if iou_score > iou_threshold:
                    expanded_slices.append((z, candidate_object_value))  # Add this slice to the object
                else:
                    break  # Stop expanding in this direction if IoU is below threshold
            else:
                break  # No object found in this slice, stop

            z += direction  # Move up or down in Z

    return label_id, expanded_slices

def resolve_conflicts(expanded_objects, membrane_filtered, membrane, centroid_dict):
    slice_ownership = {}  # Map of (z, slice) -> label ownership
    final_expanded_objects = {}

    # First pass: Assign slices to the first object that claims them
    for label_id, slices in tqdm(expanded_objects.items()):
        final_expanded_objects[label_id] = []
        for z, obj_label in slices:
            if (z, obj_label) not in slice_ownership:
                slice_ownership[(z, obj_label)] = label_id
                final_expanded_objects[label_id].append((z, obj_label))
            else:
                # Conflict! Compare IoU with current owner
                current_owner = slice_ownership[(z, obj_label)]
                current_owner_z = int(centroid_dict[current_owner][0])
                new_owner_z = int(centroid_dict[label_id][0])
                iou_current = calculate_iou(membrane_filtered[current_owner_z] == current_owner,
                                            membrane[z] == obj_label)
                iou_new = calculate_iou(membrane_filtered[new_owner_z] == label_id,
                                        membrane[z] == obj_label)

                if iou_new > iou_current:
                    # Reassign slice to the new label
                    slice_ownership[(z, obj_label)] = label_id
                    final_expanded_objects[label_id].append((z, obj_label))
                    # Remove the slice from the previous owner's list
                    if current_owner in final_expanded_objects:
                        final_expanded_objects[current_owner] = [s for s in final_expanded_objects[current_owner] if s != (z, obj_label)]

    return final_expanded_objects



In [ ]:
# Function to process a single membrane TIFF file
def process_membrane_image(file_path, output_folder, iou_threshold=0.6):
    # Read the TIFF file (membrane image)
    membrane = tiff.imread(file_path)  # shape (z, y, x)
    
    # Prepare a copy of the membrane array for filtering
    membrane_filtered = np.copy(membrane)

    # Label the membrane filtered array and extract the centroids
    labeled_membrane = label(membrane_filtered)
    object_props = regionprops(labeled_membrane)

    # Collect the centroids in a dictionary: key = label, value = (z, y, x) centroid coordinates
    centroid_dict = {}
    for prop in object_props:
        label_id = prop.label
        centroid_dict[label_id] = prop.centroid  # (z, y, x)

    z_slices = membrane.shape[0]

    # Get the number of available CPU cores
    num_cores = multiprocessing.cpu_count()

    # First parallel step - Expanding objects
    with Parallel(n_jobs=num_cores) as parallel:
        expanded_objects = parallel(
            delayed(expand_slices)(label_id, int(round(z)), int(round(y)), int(round(x)), 
                                   labeled_membrane, membrane, z_slices, iou_threshold)
            for label_id, (z, y, x) in tqdm(centroid_dict.items())
        )

    # Convert the result back to a dictionary
    expanded_objects = dict(expanded_objects)

    # Free resources and force garbage collection
    del parallel  # Explicitly remove the parallel object to free up memory
    gc.collect()  # Force garbage collection to free unused resources

    # Resolve the conflicts and get the final expanded objects
    final_expanded_objects = resolve_conflicts(expanded_objects, labeled_membrane, membrane, centroid_dict)

    # Remove entries where the value (list of slices) is empty
    final_expanded_objects = {label_id: slices for label_id, slices in final_expanded_objects.items() if slices}

    # Final membrane array to store the result
    final_membrane_array = np.zeros_like(membrane, dtype=np.int16)

    # Reconstruct the final relabelled array based on the expanded objects
    for label_id, slices in tqdm(final_expanded_objects.items()):
        for z, obj_label in slices:
            # Assign this object's label to the final membrane array at the slice
            final_membrane_array[z][membrane[z] == obj_label] = label_id

    # Apply additional filtering if necessary (filter_single_z_slice_objects function)
    final_membrane_filtered = filter_single_z_slice_objects(final_membrane_array)

    # Save the final relabelled membrane array to the output folder
    base_filename = os.path.basename(file_path)  # Get the original file name
    output_path = os.path.join(output_folder, f"final_relabelled_{base_filename}")
    tiff.imwrite(output_path, final_membrane_filtered)


# Define paths for membrane and output folders
membrane_folder = 'split_nuclei_membrane_raw/VollCellPoseSeg/CellPose/'
output_folder = 'split_nuclei_membrane_raw/split_cellpose_results/'

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Track processed files to avoid reprocessing
processed_files = set()

# Function to monitor the folder and process files as they are added
def monitor_and_process_folders():
    while True:
        # List all .tif files in the membrane folder
        current_files = sorted([f for f in os.listdir(membrane_folder) if f.endswith('.tif')])

        # Process each new file (i.e., not already in processed_files)
        for filename in current_files:
            if filename in processed_files:
                continue  # Skip if already processed

            print(f'Processing new file: {filename}')

            # Extract the time 't' value from the filename
            t = int(filename.split('-')[1].split('_cp_masks.')[0])

            input_path = os.path.join(membrane_folder, filename)
            output_filename = "final_relabelled_" + filename

            if output_filename not in os.listdir(output_folder):
                process_membrane_image(input_path, output_folder)

                # Add the processed file to the set
                processed_files.add(filename)
            else:
                print("File already in folder!")

        # If there are no new files, wait for a while before checking again
        print("No new files. Waiting for new files...")
        time.sleep(30)  # Wait for x seconds before rechecking the folder

# Run the folder monitoring function
monitor_and_process_folders()

Processing new file: Merged-100_cp_masks.tif
File already in folder!
Processing new file: Merged-101_cp_masks.tif
File already in folder!
Processing new file: Merged-102_cp_masks.tif
File already in folder!
Processing new file: Merged-103_cp_masks.tif
File already in folder!
Processing new file: Merged-104_cp_masks.tif
File already in folder!
Processing new file: Merged-105_cp_masks.tif
File already in folder!
Processing new file: Merged-106_cp_masks.tif
File already in folder!
Processing new file: Merged-107_cp_masks.tif
File already in folder!
Processing new file: Merged-108_cp_masks.tif
File already in folder!
Processing new file: Merged-109_cp_masks.tif
File already in folder!
Processing new file: Merged-10_cp_masks.tif
File already in folder!
Processing new file: Merged-110_cp_masks.tif
File already in folder!
Processing new file: Merged-111_cp_masks.tif
File already in folder!
Processing new file: Merged-112_cp_masks.tif
File already in folder!
Processing new file: Merged-113_cp_

100%|██████████| 5541/5541 [00:43<00:00, 126.03it/s]


finding labels that are only in 1 z slice
Found 3586 1 slice only labels out of 5542 total labels
Processing new file: Merged-123_cp_masks.tif


100%|██████████| 5376/5376 [00:38<00:00, 138.70it/s]


finding labels that are only in 1 z slice
Found 3426 1 slice only labels out of 5377 total labels
Processing new file: Merged-124_cp_masks.tif


...

100%|██████████| 4758/4758 [00:31<00:00, 150.77it/s]


finding labels that are only in 1 z slice
Found 3002 1 slice only labels out of 4759 total labels
Processing new file: Merged-99_cp_masks.tif


100%|██████████| 4780/4780 [00:31<00:00, 149.97it/s]


finding labels that are only in 1 z slice
Found 3049 1 slice only labels out of 4781 total labels
Processing new file: Merged-9_cp_masks.tif
File already in folder!
No new files. Waiting for new files...
Processing new file: Merged-100_cp_masks.tif
File already in folder!
Processing new file: Merged-101_cp_masks.tif
File already in folder!
Processing new file: Merged-102_cp_masks.tif
File already in folder!
Processing new file: Merged-103_cp_masks.tif
File already in folder!
Processing new file: Merged-104_cp_masks.tif
File already in folder!
Processing new file: Merged-105_cp_masks.tif
File already in folder!
Processing new file: Merged-106_cp_masks.tif
File already in folder!
Processing new file: Merged-107_cp_masks.tif
File already in folder!
Processing new file: Merged-108_cp_masks.tif
File already in folder!
Processing new file: Merged-109_cp_masks.tif
File already in folder!
Processing new file: Merged-10_cp_masks.tif
File already in folder!
Processing new file: Merged-110_cp_mas

KeyboardInterrupt: 